<a href="https://colab.research.google.com/github/JaperTai77/Colab-notebook/blob/main/pyspark_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyspark DataFrame

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 31 kB/s 
     |████████████████████████████████| 198 kB 33.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=334e718c93de10e13fa77d67c79bcb43da246d51b8a5d7ce50924fceb3ba1671
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark").getOrCreate()
spark

In [ ]:
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
cores

1

### Read files

In [ ]:
student = spark.read.csv('students.csv',inferSchema=True, header = True)

In [ ]:
student.limit(5).show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [ ]:
parquet = spark.read.parquet('users1.parquet')

In [ ]:
student.limit(5).toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [ ]:
partitioned = spark.read.parquet('users*') # all files start with users
partitioned.limit(5).show()

+-------------------+---+----------+---------+--------------------+------+--------------+----------------+------------+---------+---------+--------------------+--------+
|  registration_dttm| id|first_name|last_name|               email|gender|    ip_address|              cc|     country|birthdate|   salary|               title|comments|
+-------------------+---+----------+---------+--------------------+------+--------------+----------------+------------+---------+---------+--------------------+--------+
|2016-02-03 07:55:29|  1|    Amanda|   Jordan|    ajordan0@com.com|Female|   1.197.201.2|6759521864920116|   Indonesia| 3/8/1971| 49756.53|    Internal Auditor|   1E+02|
|2016-02-03 17:04:03|  2|    Albert|  Freeman|     afreeman1@is.gd|  Male|218.111.175.34|                |      Canada|1/16/1968|150280.17|       Accountant IV|        |
|2016-02-03 01:09:31|  3|    Evelyn|   Morgan|emorgan2@altervis...|Female|  7.161.136.94|6767119071901597|      Russia| 2/1/1960|144972.51| Structural

In [ ]:
user1_2 = spark.read.option('basePath','/content/').parquet('users1.parquet', 'users2.parquet')

### Dataframe info

In [ ]:
student.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)



In [ ]:
student.schema['math score'].dataType

IntegerType

In [ ]:
student.select('math score', 'reading score').summary('min','max').show()

+-------+----------+-------------+
|summary|math score|reading score|
+-------+----------+-------------+
|    min|         0|           17|
|    max|       100|          100|
+-------+----------+-------------+



In [ ]:
from pyspark.sql.types import *

### Custom dataframe type

In [ ]:
data_schema = [StructField("name", StringType(), True),
               StructField("email", StringType(), True),
               StructField("city", StringType(), True),
               StructField("mac", StringType(), True),
               StructField("timestamp", DateType(), True),
               StructField("creditcard", StringType(), True)]
final_struc = StructType(fields= data_schema)

people = spark.read.json('people.json', schema = final_struc)

people.limit(5).show()

+--------------------+--------------------+---------------+-----------------+----------+-------------------+
|                name|               email|           city|              mac| timestamp|         creditcard|
+--------------------+--------------------+---------------+-----------------+----------+-------------------+
|                null|                null|           null|             null|      null|               null|
|        Keeley Bosco|katlyn@jenkinsmag...|Lake Gladysberg|08:fd:0b:cd:77:f7|2015-04-25|1228-1221-1221-1431|
|         Rubye Jerde|juvenal@johnston....|           null|90:4d:fa:42:63:a2|2015-04-25|1228-1221-1221-1431|
|Miss Darian Breit...|                null|           null|f9:0e:d3:40:cb:e9|2015-04-25|               null|
|    Celine Ankunding|emery_kunze@rogah...|           null|3a:af:c9:0b:5c:08|2015-04-25|1228-1221-1221-1431|
+--------------------+--------------------+---------------+-----------------+----------+-------------------+



### Write in data

In [ ]:
student.write.mode('overwrite').csv('test.csv')
# have weird filename

In [ ]:
# create dataframe
df = spark.createDataFrame(values, columns)

### Parquet

In [ ]:
pga = spark.read.csv('pga_tour_historical.csv', schema=final_struc)
df = pga.select("Season","Value")
df.write.mode("overwrite").parquet("partition_parquet/")

dataframe = spark.read.option("basePath", path).parquet('Season=2010/','Season=2011/')
# 2010 and 2011 will be in column

# Search and Filter

In [ ]:
fifa = spark.read.csv('fifa19.csv', inferSchema = True, header = True)

In [ ]:
from pyspark.sql.functions import *

### Select dataframe

In [ ]:
fifa.select(['Nationality','Name','Age','Photo']).show(5, False) # no truncate

+-----------+-----------------+---+----------------------------------------------+
|Nationality|Name             |Age|Photo                                         |
+-----------+-----------------+---+----------------------------------------------+
|Argentina  |L. Messi         |31 |https://cdn.sofifa.org/players/4/19/158023.png|
|Portugal   |Cristiano Ronaldo|33 |https://cdn.sofifa.org/players/4/19/20801.png |
|Brazil     |Neymar Jr        |26 |https://cdn.sofifa.org/players/4/19/190871.png|
|Spain      |De Gea           |27 |https://cdn.sofifa.org/players/4/19/193080.png|
|Belgium    |K. De Bruyne     |27 |https://cdn.sofifa.org/players/4/19/192985.png|
+-----------+-----------------+---+----------------------------------------------+
only showing top 5 rows



### Orderby

In [ ]:
fifa.select(['Name', 'Age','Nationality']).orderBy(fifa['Age'].desc()).show(5)

+-------------+---+-----------------+
|         Name|Age|      Nationality|
+-------------+---+-----------------+
|     O. Pérez| 45|           Mexico|
|K. Pilkington| 44|          England|
|    T. Warner| 44|Trinidad & Tobago|
|  S. Narazaki| 42|            Japan|
|     M. Tyler| 41|          England|
+-------------+---+-----------------+
only showing top 5 rows



### Where

In [ ]:
fifa.select(['Name','Club']).where(fifa.Club.like('%Barc%')).show(5,False)

+---------------+------------+
|Name           |Club        |
+---------------+------------+
|L. Messi       |FC Barcelona|
|L. Suárez      |FC Barcelona|
|M. ter Stegen  |FC Barcelona|
|Sergio Busquets|FC Barcelona|
|Coutinho       |FC Barcelona|
+---------------+------------+
only showing top 5 rows



In [ ]:
fifa.select('Name','Club').where(fifa.Name.startswith('L')).show(5)

+----------+---------------+
|      Name|           Club|
+----------+---------------+
|  L. Messi|   FC Barcelona|
| L. Modrić|    Real Madrid|
| L. Suárez|   FC Barcelona|
|L. Insigne|         Napoli|
|   L. Sané|Manchester City|
+----------+---------------+
only showing top 5 rows



### Substr (select letters in string)

In [ ]:
fifa.select('Photo', fifa.Photo.substr(-4,4)).show(5)

+--------------------+-----------------------+
|               Photo|substring(Photo, -4, 4)|
+--------------------+-----------------------+
|https://cdn.sofif...|                   .png|
|https://cdn.sofif...|                   .png|
|https://cdn.sofif...|                   .png|
|https://cdn.sofif...|                   .png|
|https://cdn.sofif...|                   .png|
+--------------------+-----------------------+
only showing top 5 rows



### Isin

In [ ]:
fifa[fifa.Club.isin('FC Barcelona','Juventus')].show(5)

+---+------+-----------------+---+--------------------+-----------+--------------------+-------+---------+------------+--------------------+-------+-----+-------+--------------+------------------------+---------+-----------+--------------+----------+---------+--------+-------------+------------+-----------+--------------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------------+
|_c0|    ID|             Name|Age|               Photo|Nationality|                Flag|Overall|Potential|        Club|           Club Logo|  Value| Wag

In [ ]:
fifa.filter('Overall<50').select(['Name']).show(10)

+--------------+
|          Name|
+--------------+
|    D. Collins|
|       J. Egan|
|   Xie Xiaofan|
|    B. Buckley|
|  G. Figliuzzi|
|     O. Finney|
|  G. Hollywood|
|     Zhu Jiayi|
|B. Sass-Davies|
|    N. Stephan|
+--------------+
only showing top 10 rows



### Multiple operations

In [ ]:
result = fifa.filter('Overall>80').select(['Name','Age','Overall']).orderBy(fifa['Overall'].desc()).collect()

In [ ]:
result[-1]

Row(Name='Joaquín', Age=36, Overall=81)

### Spark SQL Options

In [ ]:
crime = spark.read.csv('/content/rec-crime-pfa.csv', header = True, inferSchema=True)

In [ ]:
crime.printSchema()

root
 |-- 12 months ending: string (nullable = true)
 |-- PFA: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Offence: string (nullable = true)
 |-- Rolling year total number of offences: integer (nullable = true)



In [ ]:
crime = crime.withColumnRenamed('Rolling year total number of offences', 'Count')
crime.show(5)

+----------------+-----------------+----------+--------------------+-----+
|12 months ending|              PFA|    Region|             Offence|Count|
+----------------+-----------------+----------+--------------------+-----+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|25959|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft| 3090|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|26202|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|    2|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|14561|
+----------------+-----------------+----------+--------------------+-----+
only showing top 5 rows



Use SQL language

In [ ]:
crime.createOrReplaceTempView('tempview')

In [ ]:
spark.sql("SELECT * FROM tempview").limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Count
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561


In [ ]:
result = spark.sql("SELECT Region, sum(Count) AS Total FROM tempview WHERE Count >5000 GROUP BY Region")
result.show(5)

+--------------------+--------+
|              Region|   Total|
+--------------------+--------+
|        Fraud: CIFAS| 7678981|
|          North West|25198186|
|British Transport...| 1985365|
|               Wales| 6320225|
|          South East|25246265|
+--------------------+--------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import SQLTransformer

In [ ]:
sqltrans = SQLTransformer(statement= 'SELECT PFA, offence FROM __THIS__')
sqltrans.transform(crime).show(5, False)

+-----------------+-------------------------------------------------+
|PFA              |offence                                          |
+-----------------+-------------------------------------------------+
|Avon and Somerset|All other theft offences                         |
|Avon and Somerset|Bicycle theft                                    |
|Avon and Somerset|Criminal damage and arson                        |
|Avon and Somerset|Death or serious injury caused by illegal driving|
|Avon and Somerset|Domestic burglary                                |
+-----------------+-------------------------------------------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import expr

In [ ]:
crime.withColumn('percent',expr('(count/24472098)*100')).show(5)

+----------------+-----------------+----------+--------------------+-----+--------------------+
|12 months ending|              PFA|    Region|             Offence|Count|             percent|
+----------------+-----------------+----------+--------------------+-----+--------------------+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|25959|  0.1060759073455819|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft| 3090| 0.01262662481982542|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|26202|  0.1070688749285002|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|    2|8.172572698916128E-6|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|14561|0.059500415534458874|
+----------------+-----------------+----------+--------------------+-----+--------------------+
only showing top 5 rows



In [ ]:
crime.select('*',expr('(count/24472098)*100 AS Percent')).show(5)

+----------------+-----------------+----------+--------------------+-----+--------------------+
|12 months ending|              PFA|    Region|             Offence|Count|             Percent|
+----------------+-----------------+----------+--------------------+-----+--------------------+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|25959|  0.1060759073455819|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft| 3090| 0.01262662481982542|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|26202|  0.1070688749285002|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|    2|8.172572698916128E-6|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|14561|0.059500415534458874|
+----------------+-----------------+----------+--------------------+-----+--------------------+
only showing top 5 rows



In [ ]:
crime.select('*',expr('(count/24472098)*100 AS Percent')).filter("Region = 'South West'").show(5)

+----------------+-----------------+----------+--------------------+-----+--------------------+
|12 months ending|              PFA|    Region|             Offence|Count|             Percent|
+----------------+-----------------+----------+--------------------+-----+--------------------+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|25959|  0.1060759073455819|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft| 3090| 0.01262662481982542|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|26202|  0.1070688749285002|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|    2|8.172572698916128E-6|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|14561|0.059500415534458874|
+----------------+-----------------+----------+--------------------+-----+--------------------+
only showing top 5 rows



In [ ]:
googlep = spark.read.csv("googleplaystore.csv",header=True,inferSchema=True)

In [ ]:
googlep.limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [ ]:
from pyspark.sql.types import IntegerType, FloatType
df = googlep.withColumn('Rating', googlep['Rating'].cast(FloatType())).withColumn("Reviews", googlep["Reviews"].cast(IntegerType())).withColumn("Price", googlep["Price"].cast(IntegerType()))

In [ ]:
df.createOrReplaceTempView('tempview')

In [ ]:
spark.sql('SELECT * FROM tempview WHERE Rating>4.1').show(10)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644| 25M|50,000,000+|Free|    0|          Teen|        Art & Design|      June 8, 2018|Varies with device|  4.2 and up|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|2.8M|   100,000+|Free|    0|      Everyone|Art & Design;Crea...|     June 20, 2018|               1.1|  4.4 

In [ ]:
spark.sql('SELECT Category, sum(Reviews) AS Total_reviews FROM tempview GROUP BY Category ORDER BY Total_Reviews DESC').show(10)

+---------------+-------------+
|       Category|Total_reviews|
+---------------+-------------+
|           GAME|   1585422349|
|  COMMUNICATION|    815462260|
|         SOCIAL|    621241422|
|         FAMILY|    410226330|
|          TOOLS|    273185044|
|    PHOTOGRAPHY|    213516650|
|       SHOPPING|    115041222|
|   PRODUCTIVITY|    114116975|
|  VIDEO_PLAYERS|    110380188|
|PERSONALIZATION|     89346140|
+---------------+-------------+
only showing top 10 rows



In [ ]:
spark.sql("SELECT * FROM tempview WHERE App LIKE '%dating%'").show(5, False)

+--------------------------------------------------+---------+------+-------+------------------+-----------+----+-----+--------------+----------+-------------+------------------+------------------+
|App                                               |Category |Rating|Reviews|Size              |Installs   |Type|Price|Content Rating|Genres    |Last Updated |Current Ver       |Android Ver       |
+--------------------------------------------------+---------+------+-------+------------------+-----------+----+-----+--------------+----------+-------------+------------------+------------------+
|Meet, chat & date. Free dating app - Chocolate app|DATING   |3.9   |8661   |9.5M              |1,000,000+ |Free|0    |Mature 17+    |Dating    |April 3, 2018|0.1.11            |4.0 and up        |
|Friend Find: free chat + flirt dating app         |DATING   |NaN   |23     |11M               |100+       |Free|0    |Mature 17+    |Dating    |July 31, 2018|1.0               |4.4 and up        |
|Spine- Th

# Manipulate Data

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark").getOrCreate()
spark

### Create dataframe

In [ ]:
names = spark.createDataFrame([('J','A')], ['firstn','last'])
names.show()

+------+----+
|firstn|last|
+------+----+
|     J|   A|
+------+----+



### Concat columns

In [ ]:
from pyspark.sql.functions import *
names = names.select(names.firstn,names.last,concat_ws(' ', names.firstn, names.last).alias('full'))
names.show()

+------+----+----+
|firstn|last|full|
+------+----+----+
|     J|   A| J A|
+------+----+----+



In [ ]:
names.rdd.id()

56

### Read file and transform

In [ ]:
videos = spark.read.csv('youtubevideos.csv',inferSchema=True, header = True)
videos.limit(4).toPandas()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"""last week tonight trump presidency""|""last wee...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"""rhett and link""|""gmm""|""good mythical morning""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...


In [ ]:
from pyspark.sql.types import *
df = videos.withColumn('views', videos['views'].cast(IntegerType())).withColumn("likes", videos["likes"].cast(IntegerType())).withColumn("dislikes", videos["dislikes"].cast(IntegerType())).withColumn("trending_date", to_date(videos.trending_date, 'dd.mm.yy'))

In [ ]:
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: date (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



### Replace

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df = df.withColumn('publish_time_2', regexp_replace(df.publish_time,'T',' '))
df = df.withColumn('publish_time_2', regexp_replace(df.publish_time_2,'Z',' '))
df.select('publish_time_2').show(5,False)

+------------------------+
|publish_time_2          |
+------------------------+
|2017-11-13 17:13:01.000 |
|2017-11-13 07:30:00.000 |
|2017-11-12 19:05:24.000 |
|2017-11-13 11:00:04.000 |
|2017-11-12 18:01:41.000 |
+------------------------+
only showing top 5 rows



### To datetime

In [ ]:
df = df.withColumn("publish_time_3", to_timestamp(df.publish_time_2, 'yyyy-MM-dd HH:mm:ss.SSS'))

In [ ]:
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: date (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)
 |-- publish_time_2: string (nullable = true)
 |-- publish_time_3: timestamp (nullable = true)



### Translate function

In [ ]:
df.select('publish_time', translate(col('publish_time'), 'TZ', ' ').alias('trans')).show(3,False)

+------------------------+-----------------------+
|publish_time            |trans                  |
+------------------------+-----------------------+
|2017-11-13T17:13:01.000Z|2017-11-13 17:13:01.000|
|2017-11-13T07:30:00.000Z|2017-11-13 07:30:00.000|
|2017-11-12T19:05:24.000Z|2017-11-12 19:05:24.000|
+------------------------+-----------------------+
only showing top 3 rows



### Trim

In [ ]:
df = df.withColumn('title', trim(df.title))
df.select('title').show(2)

+--------------------+
|               title|
+--------------------+
|WE WANT TO TALK A...|
|The Trump Preside...|
+--------------------+
only showing top 2 rows



### String lower

In [ ]:
df.select('title', lower(df.title)).show(5)

+--------------------+--------------------+
|               title|        lower(title)|
+--------------------+--------------------+
|WE WANT TO TALK A...|we want to talk a...|
|The Trump Preside...|the trump preside...|
|Racist Superman |...|racist superman |...|
|Nickelback Lyrics...|nickelback lyrics...|
|I Dare You: GOING...|i dare you: going...|
+--------------------+--------------------+
only showing top 5 rows



### when-otherwise 1

In [ ]:
df.select('likes','dislikes',  (when(df.likes> df.dislikes,'Good').when(df.likes<df.dislikes, 'Bad').otherwise('ND')).alias('Favor')).show(5)

+------+--------+-----+
| likes|dislikes|Favor|
+------+--------+-----+
| 57527|    2966| Good|
| 97185|    6146| Good|
|146033|    5339| Good|
| 10172|     666| Good|
|132235|    1989| Good|
+------+--------+-----+
only showing top 5 rows



### when-otherwise 2

In [ ]:
df.select('likes','dislikes', expr("CASE WHEN likes > dislikes THEN 'Good' WHEN dislikes > likes THEN 'Bad' ELSE 'UN' END AS favor")).show(3)

+------+--------+-----+
| likes|dislikes|favor|
+------+--------+-----+
| 57527|    2966| Good|
| 97185|    6146| Good|
|146033|    5339| Good|
+------+--------+-----+
only showing top 3 rows



### Concate columns

In [ ]:
df.select(concat_ws(' ',df.title, df.channel_title).alias('text')).show(2,False)

+------------------------------------------------------------------------------+
|text                                                                          |
+------------------------------------------------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE CaseyNeistat                               |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO) LastWeekTonight|
+------------------------------------------------------------------------------+
only showing top 2 rows



### Select year, month, day

In [ ]:
df.select('trending_date', year('trending_date'),month('trending_date')).show(3)

+-------------+-------------------+--------------------+
|trending_date|year(trending_date)|month(trending_date)|
+-------------+-------------------+--------------------+
|   2011-01-17|               2011|                   1|
|   2011-01-17|               2011|                   1|
|   2011-01-17|               2011|                   1|
+-------------+-------------------+--------------------+
only showing top 3 rows



In [ ]:
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: date (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)
 |-- publish_time_2: string (nullable = true)
 |-- publish_time_3: timestamp (nullable = true)



### Date difference

In [ ]:
df.select('trending_date','publish_time_3', (datediff(df.trending_date,df.publish_time_3)/365).alias('diff')).show(3)

DataFrame[trending_date: date, publish_time_3: timestamp]

### Create array of each word

In [ ]:
df.select("title",split(df.title, ' ').alias('title_array')).show(3)

+--------------------+--------------------+
|               title|         title_array|
+--------------------+--------------------+
|WE WANT TO TALK A...|[WE, WANT, TO, TA...|
|The Trump Preside...|[The, Trump, Pres...|
|Racist Superman |...|[Racist, Superman...|
+--------------------+--------------------+
only showing top 3 rows



In [ ]:
array = df.select("title",split(df.title, ' ').alias('title_array'))
array.select('title',array_contains(array.title_array, 'MARRIAGE')).show(3,False)

+--------------------------------------------------------------+-------------------------------------+
|title                                                         |array_contains(title_array, MARRIAGE)|
+--------------------------------------------------------------+-------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                            |true                                 |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)|false                                |
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons         |false                                |
+--------------------------------------------------------------+-------------------------------------+
only showing top 3 rows



### Get rid of repeat values/ Remove certian values

In [ ]:
array.select(array_distinct(array.title_array)).show(1, False)

array.select(array_remove(array.title_array, "we")).show(1, False)

+------------------------------------------+
|array_distinct(title_array)               |
+------------------------------------------+
|[WE, WANT, TO, TALK, ABOUT, OUR, MARRIAGE]|
+------------------------------------------+
only showing top 1 row

+------------------------------------------+
|array_remove(title_array, we)             |
+------------------------------------------+
|[WE, WANT, TO, TALK, ABOUT, OUR, MARRIAGE]|
+------------------------------------------+
only showing top 1 row



### Creating functions

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [ ]:
def square(x):
  return int(x**2)

In [ ]:
square_udf = udf(lambda z: square(z), IntegerType())

In [ ]:
df.select('dislikes', square_udf('dislikes')).where(col('dislikes').isNotNull()).show()

+--------+------------------+
|dislikes|<lambda>(dislikes)|
+--------+------------------+
|    2966|           8797156|
|    6146|          37773316|
|    5339|          28504921|
|     666|            443556|
|    1989|           3956121|
|     511|            261121|
|    2445|           5978025|
|     778|            605284|
|     119|             14161|
|    1363|           1857769|
|      25|               625|
|     303|             91809|
|    1333|           1776889|
|    1171|           1371241|
|     246|             60516|
|      52|              2704|
|     638|            407044|
|      53|              2809|
|      36|              1296|
|     191|             36481|
+--------+------------------+
only showing top 20 rows



# Aggregating

In [ ]:
air = spark.read.csv('nyc_air_bnb.csv', inferSchema=True, header = True)

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
air.show(4)

+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|  id|                name|host_id|  host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2539|Clean & quiet apt...|   2787|       John|           Brooklyn|   Kensington|40.64749|-73.97237|   Private room|  149|             1|                9| 2018-10-19|             0.21|                             6|             365|
|2595|Skylit Midtown Ca...|   2845|   Jennifer|          Manhatt

In [ ]:
df = air.withColumn("price", air["price"].cast(IntegerType())).withColumn("minimum_nights", air["minimum_nights"].cast(IntegerType())).withColumn("number_of_reviews", air["number_of_reviews"].cast(IntegerType())).withColumn("reviews_per_month", air["reviews_per_month"].cast(IntegerType())).withColumn("calculated_host_listings_count", air["calculated_host_listings_count"].cast(IntegerType()))

### Groupby

In [ ]:
df.groupBy('neighbourhood_group').count().show(3)

+-------------------+-----+
|neighbourhood_group|count|
+-------------------+-----+
|         Douglaston|    1|
|             Queens| 5630|
|              Nadia|    1|
+-------------------+-----+
only showing top 3 rows



In [ ]:
df.groupBy('neighbourhood_group').min('price').show(10)

+-------------------+----------+
|neighbourhood_group|min(price)|
+-------------------+----------+
|         Douglaston|         1|
|             Queens|        10|
|              Nadia|      null|
|            Midtown|         2|
|    Jackson Heights|         2|
|     Hell's Kitchen|         1|
|  Greenwich Village|        31|
|       Clinton Hill|        14|
| Washington Heights|         2|
|   Ditmars Steinway|         1|
+-------------------+----------+
only showing top 10 rows



In [ ]:
df.groupBy('neighbourhood').agg(min(df.price), max(df.price)).show(10)

+-------------+----------+----------+
|neighbourhood|min(price)|max(price)|
+-------------+----------+----------+
|       Corona|        23|       359|
| Richmondtown|        78|        78|
| Prince's Bay|        85|      1250|
|  Westerleigh|        40|       103|
|   Mill Basin|        85|       299|
|     40.76199|         1|         1|
| Civic Center|        50|       950|
|     40.83166|         1|         1|
|   Douglaston|        40|       178|
|   Mount Hope|        24|       250|
+-------------+----------+----------+
only showing top 10 rows



### Summary

In [ ]:
sum = df.summary('count','min',"25%")
sum.toPandas()

,summary,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,count,49079,49047,48894,48873,48894,48894,48894,48894,48894,48887,48891,48738,38845,38858,48891,48737
1,min,"12 mins Manhattan""",1 Bed Apt in Utopic Williamsburg,"Heart of Greenwich Village""","very clean studio app""",194716858,2,-73.72247,-73.71299,-73.90783,-74,0,0,-73.94134,0,0,0
2,25%,9470677.0,2.4544724E7,7795299.0,475.0,1.94716858E8,40.68771,40.68982,-73.9831,56.0,69,1,1,0.76,0,1,0


### Descriptive stat

In [ ]:
df.agg(min(df.price).alias('min'), max(df.price).alias('max')).show()

+---+-----+
|min|  max|
+---+-----+
|-74|10000|
+---+-----+



In [ ]:
df.select(min("neighbourhood_group").alias('CountD'),avg('price').alias('avg'),stddev("price").alias('std')).show()

+---------+------------------+-----------------+
|   CountD|               avg|              std|
+---------+------------------+-----------------+
|194716858|152.22298361527604|238.5414668883949|
+---------+------------------+-----------------+



In [ ]:
df.groupBy("neighbourhood").pivot('neighbourhood_group',['Queens','Brooklyn']).agg(min(df.price),max(df.price)).show(10)

+-------------+-----------------+-----------------+-------------------+-------------------+
|neighbourhood|Queens_min(price)|Queens_max(price)|Brooklyn_min(price)|Brooklyn_max(price)|
+-------------+-----------------+-----------------+-------------------+-------------------+
|       Corona|               23|              359|               null|               null|
| Prince's Bay|             null|             null|               null|               null|
| Richmondtown|             null|             null|               null|               null|
|   Mill Basin|             null|             null|                 85|                299|
|  Westerleigh|             null|             null|               null|               null|
|     40.76199|             null|             null|               null|               null|
| Civic Center|             null|             null|               null|               null|
|   Douglaston|               40|              178|               null|         

# Joining Appending

In [ ]:
valuesP = [('koala',1,'yes'),('caterpillar',2,'yes'),('deer',3,'yes'),('human',4,'yes')]
eats_plants = spark.createDataFrame(valuesP,['name','id','eats_plants'])
valuesM = [('shark',5,'yes'),('lion',6,'yes'),('tiger',7,'yes'),('human',4,'yes')]
eats_meat = spark.createDataFrame(valuesM,['name','id','eats_meat'])

### Appending

In [ ]:
new_df = eats_plants
df_append = eats_plants.union(new_df)
df_append.show(8)

+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
+-----------+---+-----------+



### Inner join

In [ ]:
eats_plants.join(eats_meat,['name'], 'inner').show()

+-----+---+-----------+---+---------+
| name| id|eats_plants| id|eats_meat|
+-----+---+-----------+---+---------+
|human|  4|        yes|  4|      yes|
+-----+---+-----------+---+---------+



In [ ]:
inner_join = eats_plants.join(eats_meat,['name','id'], 'inner')
inner_join.show()

+-----+---+-----------+---------+
| name| id|eats_plants|eats_meat|
+-----+---+-----------+---------+
|human|  4|        yes|      yes|
+-----+---+-----------+---------+



### Left join

In [ ]:
left_join = eats_plants.join(eats_meat,['name','id'], 'left')
left_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|caterpillar|  2|        yes|     null|
|       deer|  3|        yes|     null|
|      human|  4|        yes|      yes|
|      koala|  1|        yes|     null|
+-----------+---+-----------+---------+



### Conditional join

In [ ]:
con_join = eats_plants.join(eats_meat,['name','id'], 'left').filter(eats_meat.name.isNull())
con_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|caterpillar|  2|        yes|     null|
|       deer|  3|        yes|     null|
|      koala|  1|        yes|     null|
+-----------+---+-----------+---------+



### Full join

In [ ]:
join = eats_plants.join(eats_meat,['name','id'], 'full')
join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|caterpillar|  2|        yes|     null|
|       deer|  3|        yes|     null|
|      human|  4|        yes|      yes|
|      koala|  1|        yes|     null|
|       lion|  6|       null|      yes|
|      shark|  5|       null|      yes|
|      tiger|  7|       null|      yes|
+-----------+---+-----------+---------+



### Import data from a file

In [ ]:
import os
path='/content/'

In [ ]:
df_list = []
for f in os.listdir(path):
  if f.endswith('.csv'):
    filelist = f.split('.')
    df_name = filelist[0]
    df = spark.read.csv(path+f,inferSchema=True,header=True)
    df.name = df_name
    df_list.append(df_name)
    exec(df_name + ' = df')

### Joining real data

In [ ]:
step1 = teachings.join(instructors,teachings.instructor_id == instructors.id, 'left')
step1 = step1.select(['instructor_id','name','section_uuid'])
step1.show(5)

+-------------+----------------+--------------------+
|instructor_id|            name|        section_uuid|
+-------------+----------------+--------------------+
|       761703|JOHN ARCHAMBAULT|45adf63c-48c9-365...|
|       761703|JOHN ARCHAMBAULT|c6280e23-5e43-385...|
|       761703|JOHN ARCHAMBAULT|9395dc21-15d1-3fa...|
|      3677061|  STEPHANIE KANN|b99e440b-39db-350...|
|       761703|JOHN ARCHAMBAULT|ca1c841f-41d5-329...|
+-------------+----------------+--------------------+
only showing top 5 rows



In [ ]:
step2 = step1.join(sections, step1.section_uuid == sections.uuid, 'left').select(['name','course_offering_uuid'])
step2.limit(4).toPandas()

,name,course_offering_uuid
0,JAMES CLEARY,051f4e0c-983c-322c-a28b-51f461c662c8
1,JAMES CLEARY,051f4e0c-983c-322c-a28b-51f461c662c8
2,STEPHANIE KANN,ea3b717c-d66b-30dc-8b37-964d9688295f
3,JOHN ARCHAMBAULT,f718e6cd-33f0-3c14-a9a6-834d9c3610a8


In [ ]:
step3 = step2.withColumnRenamed('name','instructor').join(course_offerings, step2.course_offering_uuid == course_offerings.uuid,'left')
step3.show(3)

+--------------+--------------------+--------------------+--------------------+---------+--------------------+
|    instructor|course_offering_uuid|                uuid|         course_uuid|term_code|                name|
+--------------+--------------------+--------------------+--------------------+---------+--------------------+
|  JAMES CLEARY|051f4e0c-983c-322...|051f4e0c-983c-322...|9cad7b15-65ce-343...|     1122|Palliative Care (...|
|  JAMES CLEARY|051f4e0c-983c-322...|051f4e0c-983c-322...|9cad7b15-65ce-343...|     1122|Palliative Care (...|
|STEPHANIE KANN|ea3b717c-d66b-30d...|ea3b717c-d66b-30d...|a3e3e1c3-543d-3bb...|     1172|Cooperative Educa...|
+--------------+--------------------+--------------------+--------------------+---------+--------------------+
only showing top 3 rows



# Missing Value

In [ ]:
zomato = spark.read.csv('zomato.csv',inferSchema=True,header=True)

In [ ]:
zomato.printSchema()

root
 |-- url: string (nullable = true)
 |-- address: string (nullable = true)
 |-- name: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- rate: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- location: string (nullable = true)
 |-- rest_type: string (nullable = true)
 |-- dish_liked: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- approx_cost(for two people): string (nullable = true)
 |-- reviews_list: string (nullable = true)
 |-- menu_item: string (nullable = true)
 |-- listed_in(type): string (nullable = true)
 |-- listed_in(city): string (nullable = true)



In [ ]:
df = zomato.withColumn("approx_cost(for two people)", zomato["approx_cost(for two people)"].cast(IntegerType())).withColumn("votes", zomato["votes"].cast(IntegerType()))

### Filter Null

In [ ]:
df.filter(df.cuisines.isNull()).select(['name','cuisines']).show(5)

+---------------+--------+
|           name|cuisines|
+---------------+--------+
|          Jalsa|    null|
|  Grand Village|    null|
|  Casual Dining|    null|
|Timepass Dinner|    null|
|  Casual Dining|    null|
+---------------+--------+
only showing top 5 rows



### Calculate null function

In [ ]:
def nul_value_cal(df):
  null_columns_counts = []
  numRows = df.count()

  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if (nullRows>0):
      temp = k,nullRows,(nullRows/numRows)*100
      null_columns_counts.append(temp)
  return null_columns_counts

In [ ]:
nul_col_list = nul_value_cal(df)
nul_col_list

[('name', 9, 0.17311021350259664),
 ('online_order', 636, 12.233121754183497),
 ('book_table', 2, 0.03846893633391037),
 ('rate', 504, 9.694171956145412),
 ('votes', 1436, 27.62069628774764),
 ('phone', 58, 1.1155991536834007),
 ('location', 1439, 27.67839969224851),
 ('rest_type', 1442, 27.736103096749375),
 ('dish_liked', 3539, 68.0707828428544),
 ('cuisines', 1985, 38.18041931140604),
 ('approx_cost(for two people)', 2974, 57.20330832852471),
 ('reviews_list', 2064, 39.699942296595495),
 ('menu_item', 2093, 40.2577418734372),
 ('listed_in(type)', 2119, 40.75783804577803),
 ('listed_in(city)', 2147, 41.29640315445278)]

### Dropping NA

In [ ]:
df.na.drop().show(4)

+--------------------+--------------------+--------------------+------------+----------+-----+-----+--------------+------------+-------------------+--------------------+--------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+
|                 url|             address|                name|online_order|book_table| rate|votes|         phone|    location|          rest_type|          dish_liked|            cuisines|approx_cost(for two people)|        reviews_list|           menu_item|     listed_in(type)|     listed_in(city)|
+--------------------+--------------------+--------------------+------------+----------+-----+-----+--------------+------------+-------------------+--------------------+--------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+
|https://www.zomat...|2nd Floor, 80 Fee...|      Spice Elephant|         Yes|        No|4.1

In [ ]:
df.na.drop().count()/df.count()

0.12598576649355644

In [ ]:
df.na.drop(thresh = 8).count()/df.count() # does not drop all

0.9780727062896711

In [ ]:
df.na.drop(subset=['votes']).count()/df.count()

0.7237930371225235

In [ ]:
df.na.drop(how = 'all').count()/df.count() # when all columns are null

1.0

### Fill NA

In [ ]:
df.na.fill('MISSING').show(4)

+--------------------+--------------------+---------------+--------------------+--------------------+-----+-----+--------------+--------------------+-------------------+--------------------+--------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+
|                 url|             address|           name|        online_order|          book_table| rate|votes|         phone|            location|          rest_type|          dish_liked|            cuisines|approx_cost(for two people)|        reviews_list|           menu_item|     listed_in(type)|     listed_in(city)|
+--------------------+--------------------+---------------+--------------------+--------------------+-----+-----+--------------+--------------------+-------------------+--------------------+--------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+
|https://www.zomat...|942, 2

In [ ]:
df.na.fill(9999).show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-----+--------------+--------------------+-------------------+--------------------+--------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+
|                 url|             address|                name|        online_order|          book_table| rate|votes|         phone|            location|          rest_type|          dish_liked|            cuisines|approx_cost(for two people)|        reviews_list|           menu_item|     listed_in(type)|     listed_in(city)|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-----+--------------+--------------------+-------------------+--------------------+--------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+
|https://www.

In [ ]:
df.filter(df.name.isNull()).na.fill('no name', subset = ['name']).show(4)

+---------------+-----------------+-------+--------------------+--------------------+----+-----+-------------+--------------------+--------------------+--------------------+-----------------+---------------------------+-------------+--------------------+---------------+--------------------+
|            url|          address|   name|        online_order|          book_table|rate|votes|        phone|            location|           rest_type|          dish_liked|         cuisines|approx_cost(for two people)| reviews_list|           menu_item|listed_in(type)|     listed_in(city)|
+---------------+-----------------+-------+--------------------+--------------------+----+-----+-------------+--------------------+--------------------+--------------------+-----------------+---------------------------+-------------+--------------------+---------------+--------------------+
|+91 9986692090"|              BTM|no name|   Momos, Oreo Shake|Mughlai, North In...| 600| null| ('Rated 3.0'| 'RATED\n  Sim

In [ ]:
# fill with mean
def fill_with_mean(df,include = set()):
  stats = df.agg(*(avg(c).alias(c) for c in df.columns if c in include))
  return df.na.fill(stats.first().asDict())

In [ ]:
update = fill_with_mean(df,['votes'])
update.limit(5).toPandas()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555,None,None,None,None,NaN,None,None,None,None
1,"+91 9743772233""",Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,185,('Rated 4.0','RATED\n You canÃ\x83Ã\x83Ã\x82Ã\x82Ã\x...,('Rated 5.0','RATED\n Overdelighted by the service and fo...,('Rated 4.0',NaN,('Rated 4.0','RATED\n The place is nice and comfortable. ...,('Rated 4.0','RATED\n The place is nice and comfortable. ...
2,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,"""[('Rated 4.0', 'RATED\n Had been here for di...",rice was well cooked and overall was great\n\n...,('Rated 5.0','RATED\n This place just cool ? with good am...
3,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,"""[('Rated 3.0', """"RATED\n Ambience is not tha...",('Rated 3.0',"""""RATED\n \nWent there for a quick bite with ...",pasta churros and lasagne.\n\nNachos were pat...
4,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,"""[('Rated 4.0', """"RATED\n Great food and prop...",('Rated 2.0','RATED\n Reached the place at 3pm on Saturda...,('Rated 4.0'
